# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 3

## О документе

Версия 1.03

Модуль 3 для регионального этапа соревнований по кибериммунной автономности

### Модуль 3. Внедрение блоков функциональной и информационной безопасности
#### Общая идея

Ключевая идея кибериммунной автономности заключается в том, чтобы уже на этапе проектирования внедрить защиту от кибератак как снаружи, так и изнутри - т.е. через уязвимые бортовые информационные системы. 
Мы исходим из того, что абсолютно защищённых систем не бывает, чем больше и сложнее код, тем проще одним ("плохим") в нём спрятать уязвимости, а другим ("хорошим", но уставшим) эти уязвимости не найти. 

Поэтому необходимо сделать так, чтобы наиболее критичный код был отделён от некритичного и протестирован особенно тщательно - таким образом можно снизить вероятность успешной атаки и затруднить задачу атакующим.

#### Цели и предположения безопасности

##### Цели безопасности
Устойчивость кибериммунных систем к атакам выражается в том, что поставленные заказчиком **цели безопасности** (ЦБ) не нарушаются даже в условиях кибератак (снаружи и изнутри).

Для наших автономных машинок заказчиком поставлены следующие ЦБ:

1. При любых обстоятельствах АНТС осуществляет перемещения в пределах заданных ограничений

2. При любых обстоятельствах АНТС выполняет только аутентичный (подлинный) маршрут

3. При любых обстоятельствах АНТС оставляет груз только в авторизованном пункте назначения

Ниже используется цифро-буквенное обозначение целей безопасности - например, ЦБ1 - первая цель безопасности из списка выше.

##### Предположения безопасности
При этом заказчик согласовал с разработчиками следующие **предположения безопасности** (ПБ) - утверждения о смежных системах, которые снимают с разработчиков часть задач для обеспечения целей безопасности:

1. При любых обстоятельствах только авторизованный персонал имеет физический доступ к критическим узлам АНТС

2. При любых обстоятельствах только аутентичные и авторизованные операторы имеют доступ к системе планирования заданий

3. Аутентичные и авторизованные операторы обладают необходимой квалификацией и являются благонадёжными (т.е. не пытаются намеренно причинить ущерб системе или третьим лицам, используя доступ к АНТС)

Ниже используется цифро-буквенное обозначение предположений безопасности - например, ПБ1 - первое предположение безопасности из списка выше.

### Киберпрепятствия

Прежде чем мы начнём внедрять механизмы безопасности, посмотрим, что произойдёт с нашей машинкой, если бортовые системы начнут вести себя не так, как мы ожидаем.

Для этого активируем специальный режим работы системы управления и снова запустим машинку по тому же маршруту, который был в конце модуля 2.

После инициализации системы управления добавьте следующую строку
```python
control_system.enable_surprises()
```

Но вначале нужно из модуля 2 скопировать сюда ваши текущие наработки, а именно ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem), также будем использовать маршрут, который вы создали.

In [1]:
import sys
import sys
print(sys.path)



['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/user/p/.venv/lib/python3.12/site-packages']


Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [2]:
afcs_present = True

Поменяем идентификатор машинки для этого модуля

In [3]:
car_id = "m3"

In [4]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser


# возьмём маршрут из модуля 2
wpl_file = "module2.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(1, 60),
    GeoSpecificSpeedLimit(5, 20),
]


home = points[0]
mission = Mission(home=home, waypoints=points, speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

[Point(59.939106179129574, 30.315678119659424, 0.0), Point(59.93851500196842, 30.312201976776123, 0.0), Point(59.9421371401197, 30.306622982025146, 0.0), Point(59.94109460678271, 30.303940773010254, 0.0), Point(59.940374487296396, 30.302116870880127, 0.0), Point(59.94123164267495, 30.301105678081512, 0.0), Point(59.94111475915532, 30.30068188905716, 0.0)]
[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован


Итак, в отличие от, надеемся, успешного завершения маршрута в модуле 2, сейчас ваша машинка ведёт себя совсем иначе. Например, она внезапно едет в совершенно другую сторону (это особенно хорошо видно при использовании СУПА). 
И это ещё не всё, что может пойти не так..

В этой реализации не очень много кода, теоретически, вы можете его весь изучить и устранить "незадокументированные возможности", однако в реальных системах кода гораздо больше. Например, кодовая база проекта с открытым исходным кодом ArduPilot, реализующего в том числе логику автопилота, составляет примерно миллион строк. 

Сколько времени вам понадобится, чтобы изучить и протестировать миллион строк кода? А ведь придётся проводить анализ и каждого обновления.. При этом неприятности могут быть хорошо спрятанными и на первый взгляд безобидными, вряд ли злоумышленники облегчат задачу и назовут свои изменения, например, "hacking_the_system" или "allowing_secret_remote_access". Вредоносный код может выглядеть совершенно безобидно или маскироваться под программный дефект.

Поэтому тщательный анализ потребует большого количества ресурсов (человеческих, вычислительных, временныз), что стоит дорого. 

Логичным выходом является выделение критичных для целей безопасности проверок в отдельный блок, который протестировать должно быть гораздо проще.

#### Задания 

В этом модуле задание будет состоять из двух частей разной сложности:

**3.1. добавить новый блок - "Ограничитель"** - этот модуль должен будет обеспечить соблюдение ограничений, указанных в маршрутном задании - это необходимо для обеспечения ЦБ1

**З.2. *(задание повышенной сложности)* добавить монитор безопасности (МБ)** - он должен контролировать все взаимодействия *бортовых систем* (т.е. не включая планировщик и симулятор) и не давать блокам взаимодействовать друг с другом непредусмотренным образом - например, блок №5 может принимать команды только от №3, попытка передать команду в №5 от №2 должна блокироваться нашим монитором безопасности - потенциально это необходимо для обеспечения всех ЦБ

Для выполнения первой части задачи 3.1. необходимо для начала изменить архитектуру АНТС как показано на рис. 5

![Архитектура системы с ограничителем](images/ciac-arch-m31.png)

Рис. 5. Архитектура АНТС с ограничителем (для ЦБ1)

Код базового класса Ограничителя находится в файле src/safety_block.py

По аналогии с заданием в модуле 1, необходимо создать на основе этого класса свой собственный под названием SafetyBlock, в котором реализовать передачу команд в приводы движения, только перед эти необходимо ещё и проконтролировать их безопасность.

Также необходимо реализовать передачу маршрутного задания из CommunicationGateway в SafetyBlock и координат из модуля Navigation в соответствии со стрелками на рис.5 выше.

Если всё сделано правильно, то Ограничителю будет известно следующее

- _mission - текущее маршрутное задание
- _position - текущие координаты

После проверки нужно установить значение внутренних переменных
- _speed - текущая (допустимая) скорость
- _direction - текущее (допустимое) направление

Желательно завершить следование по маршруту. В критической ситуации, если продолжение маршрута невозможно, следует установить значение скорости и направления в 0, после чего передать эти команды в блок приводов (Servos)

Базовый сценарий для новой архитектуры можно представить следующим образом (см. рис. 6)

![Базовый сценарий с блоком безопасности](images/basic-scenario-with-safety-block1.png)

Рис. 6. Базовый сценарий с блоком безопасности

Заготовка кода SafetyBlock

```python
class SafetyBlock(BaseSafetyBlock):
    """ класс ограничений безопасности """

    def _set_new_direction(self, direction: float):
        """ установка нового направления перемещения """
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршруте: {self._route}")
        # TODO реализовать контроль безопасности изменения направления
        self._direction = direction
        self._send_direction_to_consumers()

    def _set_new_speed(self, speed: float):
        """ установка новой скорости """
        # TODO реализовать контроль безопасности изменения скорости
        self._speed = speed
        self._send_speed_to_consumers()


    def _send_speed_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=self._speed
                            )
        servos_q.put(event_speed)

    def _send_direction_to_consumers(self):
        self._log_message(LOG_DEBUG, "отправляем направление получателям")

        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_direction",
                            parameters=self._direction
                            )
        servos_q.put(event_speed)
```

In [5]:
from src.safety_block import BaseSafetyBlock
from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy
from src.config import (
    COMMUNICATION_GATEWAY_QUEUE_NAME, 
    CONTROL_SYSTEM_QUEUE_NAME, 
    NAVIGATION_QUEUE_NAME,
    SAFETY_BLOCK_QUEUE_NAME,
    SERVOS_QUEUE_NAME,
    CARGO_BAY_QUEUE_NAME,
    LOG_DEBUG, 
    LOG_ERROR, 
    LOG_INFO
)
from src.event_types import Event
from src.queues_dir import QueuesDirectory
from queue import Queue
import math

# ============================================================================
# ОГРАНИЧИТЕЛЬ (SAFETY BLOCK)
# ============================================================================

class SafetyBlock(BaseSafetyBlock):
    """ 
    Класс ограничителя безопасности
    
    Обеспечивает:
    - ЦБ1: соблюдение ограничений скорости и направления
    - ЦБ3: контроль доставки груза только в конечной точке маршрута
    """
    
    def _set_new_direction(self, direction: float):
        """ 
        Установка нового направления перемещения с проверкой безопасности
        
        Проверяет, что запрошенное направление соответствует расчетному
        направлению к следующей точке маршрута
        """
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршрута: {self._route}")
        
        # Если маршрут завершен, устанавливаем направление 0
        if self._route.route_finished:
            self._direction = 0
            self._send_direction_to_consumers()
            return
            
        # Рассчитываем правильное направление к следующей точке
        next_point = self._route.next_point()
        correct_direction = self._calculate_bearing(self._position, next_point)
        
        # Проверяем, что запрошенное направление близко к правильному
        # Допускаем отклонение до 5 градусов
        direction_diff = abs(direction - correct_direction)
        if direction_diff > 5 and direction_diff < 355:
            self._log_message(LOG_ERROR, 
                f"принудительная установка направления! Запрошенное направление {direction}, "
                f"расчётное {correct_direction}")
            # Устанавливаем правильное направление
            self._direction = correct_direction
        else:
            # Направление допустимо
            self._direction = direction
            
        self._send_direction_to_consumers()
    
    def _set_new_speed(self, speed: float):
        """ 
        Установка новой скорости с проверкой безопасности
        
        Проверяет, что запрошенная скорость не превышает ограничения
        для текущего сегмента маршрута
        """
        # Если маршрут завершен, устанавливаем скорость 0
        if self._route.route_finished:
            self._speed = 0
            self._send_speed_to_consumers()
            return
            
        # Получаем ограничение скорости для текущего сегмента
        speed_limit = self._route.calculate_speed()
        
        # Проверяем, что запрошенная скорость не превышает ограничение
        if speed > speed_limit:
            self._log_message(LOG_ERROR, 
                f"принудительная установка скорости! Запрошенная скорость {speed}, "
                f"ограничение {speed_limit}")
            # Устанавливаем допустимую скорость
            self._speed = speed_limit
        else:
            # Скорость допустима
            self._speed = speed
            
        self._send_speed_to_consumers()
    
    def _lock_cargo(self, _):
        """ 
        Блокировка грузового отсека
        
        Разрешаем блокировку в любой момент
        """
        self._send_lock_cargo_to_consumers()
    
    def _release_cargo(self, _):
        """ 
        Разблокировка грузового отсека
        
        Разрешаем только в конечной точке маршрута
        """
        if self._route.route_finished:
            self._log_message(LOG_INFO, "маршрут завершен, разрешаем выгрузку груза")
            self._send_release_cargo_to_consumers()
        else:
            self._log_message(LOG_ERROR, "попытка выгрузки груза вне конечной точки маршрута!")
    
    def _send_speed_to_consumers(self):
        """ Отправка команды скорости в сервоприводы """
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # Отправка сообщения с допустимой скоростью
        event_speed = Event(
            source=self.event_source_name,
            destination=servos_q_name,
            operation="set_speed",
            parameters=self._speed
        )
        servos_q.put(event_speed)

    def _send_direction_to_consumers(self):
        """ Отправка команды направления в сервоприводы """
        self._log_message(LOG_DEBUG, "отправляем направление получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # Отправка сообщения с допустимым направлением
        event_direction = Event(
            source=self.event_source_name,
            destination=servos_q_name,
            operation="set_direction",
            parameters=self._direction
        )
        servos_q.put(event_direction)
        
    def _send_lock_cargo_to_consumers(self):
        """ Отправка команды блокировки груза """
        self._log_message(LOG_DEBUG, "отправляем команду блокировки груза")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q: Queue = self._queues_dir.get_queue(cargo_q_name)

        # Отправка сообщения о блокировке груза
        event_lock = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="lock_cargo",
            parameters=None
        )
        cargo_q.put(event_lock)
        
    def _send_release_cargo_to_consumers(self):
        """ Отправка команды разблокировки груза """
        self._log_message(LOG_DEBUG, "отправляем команду разблокировки груза")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q: Queue = self._queues_dir.get_queue(cargo_q_name)

        # Отправка сообщения о разблокировке груза
        event_release = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="release_cargo",
            parameters=None
        )
        cargo_q.put(event_release)
        
    def _calculate_bearing(self, start, end):
        """ 
        Расчет направления (азимута) от начальной точки к конечной
        
        Args:
            start: начальная точка (GeoPoint)
            end: конечная точка (GeoPoint)
            
        Returns:
            float: направление в градусах (0-360)
        """
        delta_longitude = end.longitude - start.longitude
        x = math.sin(math.radians(delta_longitude)) * math.cos(math.radians(end.latitude))
        y = math.cos(math.radians(start.latitude)) * math.sin(math.radians(end.latitude)) - \
            math.sin(math.radians(start.latitude)) * math.cos(math.radians(end.latitude)) * \
            math.cos(math.radians(delta_longitude))
        
        initial_bearing_rad = math.atan2(x, y)
        initial_bearing_deg = math.degrees(initial_bearing_rad)
        compass_bearing = (initial_bearing_deg + 360) % 360
        
        return compass_bearing

In [6]:
from src.config import (
    CONTROL_SYSTEM_QUEUE_NAME, 
    SAFETY_BLOCK_QUEUE_NAME,
    SERVOS_QUEUE_NAME,
    CARGO_BAY_QUEUE_NAME,
    LOG_ERROR, 
    LOG_INFO,
    SECURITY_MONITOR_QUEUE_NAME
)
from src.event_types import Event
from src.queues_dir import QueuesDirectory
from queue import Queue
from src.control_system import BaseControlSystem
from src.navigation_system import BaseNavigationSystem
from src.communication_gateway import BaseCommunicationGateway

# ============================================================================
# КОММУНИКАЦИОННЫЙ ШЛЮЗ (COMMUNICATION GATEWAY)
# ============================================================================

class CommunicationGateway(BaseCommunicationGateway):
    """ 
    Класс коммуникационного шлюза
    
    Отправляет маршрутное задание в систему управления через монитор безопасности
    """
    
    def _send_mission_to_consumers(self):
        """ 
        Отправка маршрутного задания получателям через монитор безопасности
        """
        # Создаем событие для системы управления
        self._log_message(1, 'Отправлен Евент')
        event_safety = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )

        event_control = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )
        
        safety_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        safety_q.put(event_safety)
        control_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        control_q.put(event_control)
        
        #security_monitor_q: Queue = self._queues_dir.get_queue(SAFETY_BLOCK_QUEUE_NAME)
        #security_monitor_q.put(event)


# ============================================================================
# СИСТЕМА НАВИГАЦИИ (NAVIGATION SYSTEM)
# ============================================================================

class NavigationSystem(BaseNavigationSystem):
    """ 
    Класс навигационного блока
    
    Отправляет координаты в систему управления через монитор безопасности
    """
    
    def _send_position_to_consumers(self):
        """ 
        Отправка координат получателям через монитор безопасности
        """
        # Создаем событие для системы управления
        event_control = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )
        
        event_safety = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )

        # Отправляем событие через монитор безопасности
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event_safety)
        security_q.put(event_control)

# ============================================================================
# СИСТЕМА УПРАВЛЕНИЯ (CONTROL SYSTEM)
# ============================================================================

class ControlSystem(BaseControlSystem):
    """ 
    Класс системы управления
    
    Отправляет команды скорости и направления в ограничитель через монитор безопасности
    """
    
    def _send_speed_and_direction_to_consumers(self, speed: float, direction: float):
        """ 
        Отправка команд скорости и направления в ограничитель через монитор безопасности
        """
        # Создаем события для ограничителя
        speed_event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_speed",
            parameters=speed
        )
        
        direction_event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_direction",
            parameters=direction
        )
        
        # Отправляем события через монитор безопасности
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(speed_event)
        security_monitor_q.put(direction_event)
    
    def _release_cargo(self):
        """ 
        Отправка команды разблокировки груза в ограничитель через монитор безопасности
        """
        # Создаем событие для ограничителя
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="release_cargo",
            parameters=None
        )
        
        # Отправляем событие через монитор безопасности
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
    
    def _lock_cargo(self):
        """ 
        Отправка команды блокировки груза в ограничитель через монитор безопасности
        """
        # Создаем событие для ограничителя
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="lock_cargo",
            parameters=None
        )
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
    
        
navigation_system = NavigationSystem(queues_dir)
control_system = ControlSystem(queues_dir)
communication_gateway = CommunicationGateway(queues_dir)

[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи


In [7]:
# создадим все остальные компоненты

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_DEBUG)

[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель


#### Обеспечение ЦБ3: контроль доставки груза

С помощью логики ограничителя выше мы реализовали обеспечение ЦБ1 (соблюдение заданных ограничений), однако есть ещё ЦБ3 (отгрузка только в заданной точке), не говоря о ЦБ2 (следование только по аутентичному маршруту).

Для обеспечения ЦБ3 необходимо сделать следующее
1. изменить последовательность передачи команд, чтобы в блок управления грузовым отсеком (CargoBay) команды приходили только после проверки в Ограничителе
2. доработайте реализацию Ограничителя, чтобы команда на выгрузку передавалась в CargoBay только в конечной точке маршрута
3. продумайте логику работы блока безопасности, если система управления полностью перестанет работать (т.е. перестанет отправлять новые команды).

Изменённая архитектура приведена на рис. 7.

![Ограничитель для обеспечения ЦБ1 и ЦБ3](images/ciac-arch-m312.png)

Рис. 7. Ограничитель для обеспечения ЦБ1 и ЦБ3

In [8]:
# напишите свой код в этой ячейке, запустите машинку по маршруту, убедитесь, 
# что в конечной (и только в конечной!) точке CargoBay получает команду и успешно выполняет выгрузку

#### *Опциональное задание повышенной сложности* Обеспечение ЦБ2

**Перед началом работы над этой задачей рекомендуем сохранить все текущие изменения в git репозитории (git commit, git push).**

Напомним цель безопасности №2 (ЦБ2): "При любых обстоятельствах АНТС выполняет только аутентичный (подлинный) маршрут"

Обеспечение ЦБ2 потребует 
- внедрения механизма **контроля аутентичности маршрутного задания** (mission), а значит
- помимо изменения **блока безопасности** (SafetyBlock), модификации 
  - системы **планирования задания** (MissionPlanner)
  - **коммуникационного шлюза** (CommunicationGateway)
  - и, возможно, **системы управления** (ControlSystem). 
 
Для решения этой задачи допустимо менять код соответствующих модулей в папке src.

Возможным решением является внедрение механизма цифровой подписи, верификацией которой будет заниматься блок безопасности. В этой ситуации если коммуникационный шлюз (блок "1. Связь" на рис. 5 в модуле 1) оказался скомпрометированным и изменил маршрутное задание, оно не должно пройти проверку подлинности в блоке безопасности.

**Рекомендуем эту задачу делать в последнюю очередь**, *после успешного выполнения модуля 4*, поскольку успешное выполнение модуля 4 принесёт больше баллов, чем внедрение цифровой подписи и обеспечение ЦБ2. 
К сведению, реализованные в этой версии киберпрепятствия не предполагают компрометацию маршрутного задания.

Конечно, если будут решены все основные задачи и к тому же обеспечен контроль аутентичности маршрутного задания, такая работа с высокой вероятностью окажется лучшей.

### Задание 3.2. Монитор безопасности

В текущей реализации каждый блок сам решает куда и какие данные ему отправлять. Это приемлемо, когда каждый блок рассматривается как благонадёжный. Однако нельзя исключать ситуацию успешной компрометации какого-либо из блоков (одного или нескольких), в результате которой злоумышленник может получить полный контроль над частью системы. 

В такой ситуации необходимо выстроить внутреннюю "поверхность защиты", частью этой системы уже является блок "Ограничитель", однако необходимо сделать так, чтобы соответствующие запросы проходили только через него, а не в обход.

Для решения этой задачи необходимо добавить новый блок "Монитор безопасности". Его задача - обеспечить контроль направления информационных потоков, допустить передачу данных только между теми блоками и в тех направлениях, которые задуманы архитектором системы. Тогда, например, попытка передать команду из системы управления напрямую в сервоприводы в обход ограничителя будет заблокирована.

Возможно, возникнет вопрос, а что помешает злоумышленнику отправить команду в обход этого монитора безопасности. В рамках этой учебной инфраструктуры особо ничего, однако при разработке реальной системы можно использовать специализированные микроядерные операционные системы, такие как, например, KasperskyOS, которые обеспечат обязательное вовлечение монитора безопасности в контроль взаимодействия между блоками.

Архитектура системы для решения задачи 3.2 должна выглядеть как на рис.8:

![Архитектура системы с монитором безопасности](images/ciac-arch-m32.png)

Рис. 8. Архитектура бортовых систем с монитором безопасности

Базовый класс модуля безопасности находится в файле src/security_monitor.py, его менять не нужно. 

В следующей ячейке находится реализация класса SecurityMonitor

In [9]:
### Киберпрепятствия
from src.queues_dir import QueuesDirectory
from src.security_policy_type import SecurityPolicy

from src.config import COMMUNICATION_GATEWAY_QUEUE_NAME, LOG_DEBUG, LOG_ERROR, LOG_INFO
from src.security_monitory import BaseSecurityMonitor
from src.event_types import Event
    
    
class SecurityMonitor(BaseSecurityMonitor):
   
    def __init__(self, queues_dir: QueuesDirectory):
        """
        Иницилиация полити безопаснсоти по умолчанию

        Переопрделение типа переменной self__security_policies с словаря на список
        """
        super().__init__(queues_dir)
        self._security_policies = set()
        self._init_set_security_policies()
     
    
    def _init_set_security_policies(self):
        """ 
        Инициализация политик безопасности
        
        Определяет разрешенные взаимодействия между компонентами системы
        """
        # Базовые политики безопасности
        default_policies = [
            # Коммуникационный шлюз -> Система управления: установка маршрутного задания
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'),

            # Коммуникационный шлюз -> Огрначительный блок: установка маршрутного задания
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_mission'), 

            # Система управления -> Ограничитель: установка скорости
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_speed'),
                
            # Система управления -> Ограничитель: установка направления
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_direction'),

            # Система Навигации -> Ограничетльный блок: установка направления
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='position_update'),

            # Система навигации -> Блок Управления: установка направления
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='position_update'),

            # Система управления -> Ограничитель: блокировка груза
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='lock_cargo'),
                
            # Система управления -> Ограничитель: разблокировка груза
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='release_cargo'),
                
            # Ограничитель -> Сервоприводы: установка скорости
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_speed'),
                
            # Ограничитель -> Сервоприводы: установка направления
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_direction'),
                
            # Ограничитель -> Грузовой отсек: блокировка груза
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='lock_cargo'),
                
            # Ограничитель -> Грузовой отсек: разблокировка груза
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='release_cargo'),
        ]
        
        self.set_security_policies(policies=default_policies)
    
    def set_security_policies(self, policies):
        """ 
        Установка новых политик безопасности
        
        Args:
            policies: список политик безопасности
        """
        self._security_policies = policies
        self._log_message(
            LOG_INFO, f"изменение политик безопасности: {policies}")
    
    def _check_event(self, event: Event):
        """ 
        Проверка входящих событий на соответствие политикам безопасности
        
        Args:
            event: проверяемое событие
            
        Returns:
            bool: True если событие разрешено, False если запрещено
        """
        self._log_message(
            LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")
        
        authorized = False
     
        # Проверяем, есть ли соответствующая политика
        for policy in self._security_policies:
            if (policy.source == event.source and 
                policy.destination == event.destination and 
                policy.operation == event.operation):
                self._log_message(
                    LOG_DEBUG, "событие разрешено политиками, выполняем")
                authorized = True
                break
                
            self._log_message(
                LOG_DEBUG, "событие разрешено политиками, выполняем")
        
        if authorized is False:
            self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
            
        return authorized 
    

    # соберём все компоненты для запуска

security = SecurityMonitor(queues_dir)
system_components = SystemComponentsContainer(
components=[
        # вариант компонентов с передачей телеметрии в СУПА
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security
    ] if afcs_present else [
        # вариант компонентов для конфигурации без СУПА
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security
    ])

system_components.start()
control_system.enable_surprises()

# настройте этот параметр так, чтобы проверить работу ограничителя
sleep(60)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] регистрируем очередь security
[ИНФО][SECURITY] создан монитор безопасности
[ИНФО][SECURITY] изменение политик безопасности: [SecurityPolicy(source='communication', destination='control', operation='set_mission'), SecurityPolicy(source='communication', destination='safety', operation='set_mission'), SecurityPolicy(source='control', destination='safety', operation='set_speed'), SecurityPolicy(source='control', destination='safety', operation='set_direction'), SecurityPolicy(source='navigation', destination='safety', operation='position_update'), SecurityPolicy(source='navigation', destination='control', operation='position_update'), SecurityPolicy(source='control', destination='safety', operation='lock_cargo'), SecurityPolicy(source='control', destination='safety', operation='release_cargo'), SecurityPolicy(source='safety', destination='servos', operation='set_speed'), SecurityPolicy(source='safety', destination='servos', operation='set_direction'), SecurityPolicy(source='

[ИНФО][MISSION PLANNER] старт системы планирования заданий[ИНФО][NAVIGATION] старт навигации

[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][COMMUNICATION] старт системы планирования заданий[ИНФО][CONTROL] старт системы управления

[ИНФО][SAFETY] старт ограничителя
[ИНФО][SECURITY] старт блока грузового отсека
59 56m 20.7822s N, 30 18m 56.4412s E
59 56m 18.654s N, 30 18m 43.9271s E
59 56m 31.6937s N, 30 18m 23.8427s E
59 56m 27.9406s N, 30 18m 14.1868s E
59 56m 25.3482s N, 30 18m 7.62074s E
59 56m 28.4339s N, 30 18m 3.98044s E
59 56m 28.0131s N, 30 18m 2.4548s E
Mission(home=Point(59.939106179129574, 30.315678119659424, 0.0), waypoints=[Point(59.939106179129574, 30.315678119659424, 0.0), Point(59.93851500196842, 30.312201976776123, 0.0), Point(59.9421371401197, 30.306622982025146, 0.0), Point(59.94109460678271, 30.303940773010254, 0.0), Point(59.940374487296396, 30.302116870880127, 0.0), Point(59.94123164267495, 30.301105678081512, 0.0), Point(59.94111475915532, 30.30068188905716, 0

Process CargoBay-15:
Process MissionPlanner-12:
Process CommunicationGateway-9:
Process MissionSender-10:
Process SafetyBlock-16:
Process ControlSystem-8:
Process NavigationSystem-7:
Process Servos-14:
Process TelemetrySender-11:
Process SITL-13:
Process SecurityMonitor-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/mnt/duma2/src/communication_gateway.py", line 110, in run
    sleep(self._recalc_interval_sec)
  File "/mnt/duma2/src/navi

KeyboardInterrupt: 

Обратите внимание на этот код:
```python
    def _init_set_security_policies(self):
        """ инициализация политик безопасности """
        default_policies = [
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission')
        ]
        self.set_security_policies(policies=default_policies)  
```

В нём инициализируется политика безопасности, разрешающая запрос set_mission от коммуникационного шлюза в систему управления.

Код политики безопасности в строка 6-9:
```python
SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission')
```

Шаблон политики безопасности описан в файле src/security_policy_type.py:
```python
@dataclass
class SecurityPolicy:
    """ политика безопасности """
    source: str         # отправитель запроса
    destination: str    # получатель
    operation: str      # запрашиваемая операция

```

По аналогии в методе _init_set_security_policies допишите другие политики безопасности, чтобы разрешить взаимодействия согласно диаграмме на рис.7. выше. 

Теперь необходимо изменить взаимодействие между блоками так, чтобы оно всегда проходило через монитор безопасности. Для этого потребуется изменить соответствующие реализации внутренних блоков.

Ниже пример для блока навигации. Обратите внимание на выделенные строки и комментарии.

```python
from src.navigation_system import BaseNavigationSystem
from src.config import CONTROL_SYSTEM_QUEUE_NAME, SECURITY_MONITOR_QUEUE_NAME


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):
        # событие для системы управления,
        # поэтому в destination указываем CONTROL_SYSTEM_QUEUE_NAME        
        event = Event(source=self.event_source_name,
>>>                      destination=CONTROL_SYSTEM_QUEUE_NAME,
                      operation="position_update", parameters=self._position
                      )
        # но событие отправляем сначала на проверку в монитор безопасности,
        # поэтому используем очередь с именем SECURITY_MONITOR_QUEUE_NAME
>>>        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
        # а в систему управления наше событие монитор безопасности отправит сам, если найдёт разрешающую эту операцию политику безопасности

navigation_system = NavigationSystem(queues_dir=queues_dir)
```

from src.navigation_system import BaseNavigationSystem
from src.config import CONTROL_SYSTEM_QUEUE_NAME, SECURITY_MONITOR_QUEUE_NAME


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):        
        event = Event(source=self.event_source_name,
                      destination=CONTROL_SYSTEM_QUEUE_NAME,
                      operation="position_update", parameters=self._position
                      )
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)

navigation_system = NavigationSystem(queues_dir=queues_dir)

В файле tests/module/test_security_monitor.py можно увидеть тест логики работы монитора безопасности. Запустить его можно следующей командой:

In [ ]:
!pytest -svk security_monitor

В одном кодовом блоке ниже (или нескольких - как будет удобнее)  сделайте следующее

1. Измените реализацию монитора безопасности в части правил проверки передаваемых сообщений в мониторе безопасности
2. Измените параметры отправки сообщений между элементами бортовых систем (блоками 1-7), чтобы они всегда отправлялись в очередь монитора безопасности (отправитель и получатель при этом не меняются)
3. После всех изменений убедитесь в том, что машинка по-прежнему успешно выполняет маршрутное задание и доставляет груз
4. Дополнительные баллы принесут автоматические тесты правил безопасности в файле tests/module/test_security_monitor.py - можно проверить, что все новые политики работают как ожидаются, а недопустимые взаимодействия (т.е. таких стрелок нет на рис. 8) блокируются

In [ ]:
# добавьте изменения сюда

На этом модуль 3 завершён! 

Если у вас всё получилось, то вы создали защищённую автономную машинку - ещё уже не так-то просто использовать для нанесения критического ущерба людям и имуществу, потому что самые критические режимы проверяются вашим Ограничителем, а если вы ещё и внедрили Монитор Безопасности, то теперь злоумышленнику гораздо сложнее атаковать внутренние блоки, даже если он смог проникнуть в какую-то бортовую систему или внедрить вредоносный код.

Осталось проверить эффективность получившейся защиты на полной трассе - в этом и заключается модуль 4 задания, для перехода к которому нужно открыть блокнот с именем cyberimmunity--autonomous-car-m4. Успеха!